<a href="https://colab.research.google.com/github/JiaweiYu1/fasttext/blob/master/fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import re
import os
import codecs

import tensorflow as tf

import time
import datetime

from sklearn.utils import shuffle
import csv
import html

from tensorflow.contrib import slim

import argparse
from datetime import datetime
import pickle
import pprint
import easydict
import random


#tf.enable_eager_execution()

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Colab_Notebooks/fasttext")
cwd = os.getcwd()
print(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/fasttext


In [0]:
class FastText:
  
  def __init__(self,
               class_size, sentence_length, vocab_size,
               embedding_dim, learning_rate, num_sampled, epoch):
    self.class_size = class_size
    self.sentence_length = sentence_length
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.learning_rate = learning_rate
    self.num_sampled = num_sampled
    self.epoch = epoch
    
    self.global_step = tf.Variable(0, name="global_step", trainable=False)
    self.input_x = tf.placeholder(tf.int32, [None, self.sentence_length], name= 'input_x')
    self.input_y = tf.placeholder(tf.int32, [None], name='input_y')
    #self.loss = tf.constant(0.0)
    #self.global_step = tf.Variable(0, trainable=False, name="Global_Step")
    
    self.model()
    #self.instantiate_weights()
    #self.logits = self.inference()
    
    #self.loss_val = self.loss()
    #self.train_op = self.train()
    
    #self.accuracy = self.get_accuracy()
    #self.predictions = tf.argmax(self.logits, axis=1, name="predictions")  # shape:[None,]
    #correct_prediction = tf.equal(tf.cast(self.predictions, tf.int32), self.input_y)
    #self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="Accuracy")
    
  
  def model(self):
    
    with tf.variable_scope('embedding', reuse=tf.AUTO_REUSE):
      self.Embedding = tf.get_variable('embedding',[self.vocab_size, self.embedding_dim])
      self.W = tf.get_variable('W', [self.embedding_dim, self.class_size])
      self.b = tf.get_variable('b',[self.class_size])
      
      self.embedding_chars = tf.nn.embedding_lookup(self.Embedding, self.input_x)
      
      self.sentence_embeddings = tf.reduce_mean(self.embedding_chars, axis=1)
      self.logits = tf.matmul(self.sentence_embeddings, self.W) + self.b
      
      input_y = tf.reshape(self.input_y, [-1])
      input_y = tf.expand_dims(input_y, 1)
      self.loss = tf.reduce_mean(
                 tf.nn.nce_loss(weights=tf.transpose(self.W),
                               biases=self.b,
                               labels=input_y,
                               inputs=self.sentence_embeddings,
                               num_sampled=self.num_sampled,
                               num_classes=self.class_size,
                               partition_strategy="div"))

    #with tf.name_scope('opt'):
      #optimizer= tf.train.AdamOptimizer(learning_rate)
      #update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
      #self.train_op = slim.learning.create_train_op(total_loss=self.loss, optimizer=optimizer,update_ops=update_ops)
      self.train_op = tf.contrib.layers.optimize_loss(self.loss, global_step=self.global_step, learning_rate=self.learning_rate, optimizer="Adam")
      
    #with tf.name_scope('accuracy'):
      self.predictions = tf.argmax(self.logits, axis = 1, name='predictions')
      correct_prediction = tf.equal(tf.cast(self.predictions, tf.int32), self.input_y)
      self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='Accuracy')
      
  def fit(self, train_x, train_y, batch_size):
    if not os.path.exists('saves/fasttext'): os.makedirs('saves/fasttext')
    if not os.path.exists('train_logs/fasttext'): os.makedirs('train_logs/fasttext')

    train_steps = 0
    best_val_acc = 0
    
    tf.summary.scalar('val_loss', self.loss)
    tf.summary.scalar('val_accuracy', self.accuracy)
    merged = tf.summary.merge_all()
    
    sess = tf.Session()
    
    writer = tf.summary.FileWriter('train_logs/fasttext', sess.graph)
    saver = tf.train.Saver(max_to_keep=10)
    sess.run(tf.global_variables_initializer())
    
    for i in range(self.epoch):
      batch_train = self.batch_iter(train_x, train_y, batch_size)
      for batch_x, batch_y in batch_train:
        train_steps +=1
        feed_dict = {self.input_x:batch_x, self.input_y:batch_y}
        _, train_loss, train_acc = sess.run([self.train_op, self.loss, self.accuracy], feed_dict = feed_dict)
        
        if train_steps % 1000 ==0:
          #feed_dict = {self.input_x:val_x, self.input_y:val_y}
          val_loss, val_acc = sess.run([self.loss, self.acc],feed_dict=feed_dict)
        
          summary = sess.run(merged, feed_dict=feed_dict)
          writer.add_summary(summary, global_step=train_steps)
        
          if val_acc >= best_val_acc:
            best_val_acc = val_acc
            saver.save(sess, "saves/fasttext", global_step=train_steps)
          msg = 'epoch:%d/%d, train_steps:%d, train_loss:%.4f, trina_acc:%.4f, val_loss:%.4f, val_acc:%.4f'
          print(msg%(i, self.epoch, train_steps, train_loss, train_acc, val_loss, val_acc))
        
  def batch_iter(self, x, y, batch_size=1, shuffle=True):
    data_len = len(x)
    num_batch = int((data_len-1)/batch_size)+1
    
    if shuffle:
      #shuffle_indices = np.random.permutation(np.arange(data_len))
      #x_shuffle = x[shuffle_indices]
      #y_shuffle= y[shuffle_indices]
      random.shuffle(x)
      x_shuffle = x
      random.shuffle(y)
      y_shuffle = y
    else:
      x_shuffle=x
      y_shuffle = y
    for i in range(num_batch):
      start_index = i*batch_size
      end_index = min((i+1)*batch_size, data_len)
      yield (x_shuffle[start_index:end_index], y_shuffle[start_index:end_index])
      
  def predict(self, x):
    sess = tf.Session()
    
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Svaer(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state('')
    saver.restore(sess, ckpt.model_checkpoint_path)
    
    feed_dict={self.input_x:x}
    logits = sess.run(self.logits, feed_dict=feed_dict)
    y_pred = np.argmax(logits, 1)
    
    return y_pred
  #def instantiate_weights(self):
  #  with tf.name_scope('Embedding'):
  #    self.Embedding = tf.Variable([self.vocab_size, self.embedding_dim], name='Embedding')
  #    self.W = tf.get_variable('W', [self.embedding_dim, self.class_size])
  #    self.b = tf.get_variable('b', [self.class_size])
    
  #def inference(self):
  #  embedding_chars = tf.nn.embedding_lookup(self.Embedding, self.input_x)
  #  self.sentence_embeddings = tf.reduce_mean(embedding_chars, axis =1)
  #  logits = tf.matmul(self.sentence_embeddings, self.W)+self.b
  #  return logits
  
  #def loss(self):
  #  input_y = tf.reshape(self.input_y, [-1])
  #  input_y = tf.expand_dims(input_y, 1)
  #  loss = tf.reduce_mean(
  #         tf.nn.nce_loss(weights=tf.transpose(self.W),
  #                             biases=self.b,
  #                             labels=input_y,
  #                             inputs=self.sentence_embeddings,
  #                             num_sampled=10,
  #                             num_classes=self.class_size,
  #                             partition_strategy="div"))
  #  return loss
  
  #def train(self):
  #      train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step, learning_rate=self.learning_rate, optimizer="Adam")
  #      return train_op
    
  #def get_accuracy(self):
  #  self.predictions = tf.argmax(self.logits, axis =1, name='predictions')
  #  correct_preidctions = tf.equal(self.predictions, self.input_y)
  #  accruacy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
  #  return accuracy
      
      
      
    
    
    

In [0]:

#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.0.0/en_core_web_lg-2.0.0.tar.gz

In [5]:
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load('en_core_web_lg')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
def get_ngram(words, n=2):
    # TODO add ngrams up to n
    return [' '.join(words[i: i+n]) for i in range(len(words)-(n-1))]

In [0]:
class load_data:
  
  def __init__(self, config):
    self.config = config
    self.class_size = config.class_size
    self.max_len = config.max_len
    self.n_over_max_len = 0
    self.real_max_len = 0
    
    np.random.seed(config.seed)
    
    self.ngram2idx = dict()
    self.idx2ngram = dict()
    self.ngram2idx['PAD'] = 0
    self.idx2ngram[0] = 'PAD'
    self.ngram2idx['UNK'] = 1
    self.idx2ngram[1] = 'UNK'
    self.html_tag_re = re.compile(r'<[^>]+>')
    self.train_data, self.test_data = self.load_csv()
    self.train_data, self.valid_data = \
        self.split_tr_va(n_class_examples=config.valid_size_per_class)
    self.count_labels()

    print('real_max_len', self.real_max_len)
    print('n_over_max_len {}/{} ({:.1f}%)'.
          format(self.n_over_max_len, len(self.train_data),
          100 * self.n_over_max_len / len(self.train_data)))
    
  def load_csv(self):
    train_data = list()
    test_data = list()
    
    #spacy.prefer_gpu()

    # https://spacy.io/usage/facts-figures#benchmarks-models-english
    # python3 -m spacy download en_core_web_lg --user
    nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner'])
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    
    with open(self.config.train_data_path, 'r', newline='',
              encoding='utf-8') as f:
      reader = csv.reader(f, quotechar='"')
      for idx, features in enumerate(reader):
        y = int(features[0]) -1
        assert 0 <= y <= self.class_size, y
        x, x_len = self.process_example(features[1], features[2], nlp, is_train=True,
                                       padding=args.padding)
        train_data.append([x, x_len, y])
        
        if (idx+1) %10000 ==0:
          print(idx+1)
    
    with open(self.config.test_data_path,'r', newline='',
             encoding = 'utf-8') as f:
      reader = csv.reader(f, quotechar='"')
      for idx, features in enumerate(reader):
        y = int(features[0]) -1
        assert 0 <= y <= self.class_size, y
        x, x_len = self.process_example(features[1], features[2], nlp, is_train=True,
                                       padding=args.padding)
        test_data.append([x, x_len, y])
        
    print('dictionary size ', len(self.ngram2idx))
    
    return train_data, test_data
  
  def process_example(self, title, description, nlp, is_train=True, padding=0):
    title_desc = title + '. ' + description
    
    if '\\' in title_desc:
      title_desc = title_desc.replace('\\',' ')
      
    title_desc = html.unescape(title_desc)
    
    if '<' in title_desc and '>' in title_desc:
      title_desc = self.html_tag_re.sub('', title_desc)
      
    # create bow and bag-of-ngrams
    doc = nlp(title_desc)
    b_o_w = [token.text for token in doc]
    
    
    # add tags for ngrams
    tagged_title_desc = \
    '<p> ' + ' </s>'.join([s.text for s in doc.sents])+\
    ' <p>'
    doc = nlp(tagged_title_desc)
    n_gram = get_ngram([token.text for token in doc],
                       n = self.config.n_gram)
    
    b_o_ngrams = b_o_w+n_gram
    
    if padding >0:
      if self.max_len < len(b_o_ngrams):
        b_o_ngrams = b_o_ngrams[:self.max_len]
        
    if is_train:
      for ng in b_o_ngrams:
        idx = self.ngram2idx.get(ng)
        if idx is None:
          idx = len(self.ngram2idx)
          self.ngram2idx[ng] = idx
          self.idx2ngram[idx] = ng
          
    x = [self.ngram2idx[ng] if ng in self.ngram2idx
         else self.ngram2idx['UNK'] for ng in b_o_ngrams]
    
    x_len = len(x)
    
    if x_len > self.max_len:
      self.n_over_max_len +=1
      
    if x_len > self.real_max_len:
      self.real_max_len = x_len
      
    if padding >0:
      while len(x) < self.max_len:
        x.append(self.ngram2idx['PAD'])
      assert len(x) == self.max_len
    return x, x_len
  
  def count_labels(self):
    def count(data):
      count_dict = dict()
      for d in data:
        if d[-1] not in count_dict:
          count_dict[d[-1]] = 1
        else:
          count_dict[d[-1]] +=1
          
      return count_dict
    print('train', count(self.train_data))
    print('test', count(self.test_data))
    
  def split_tr_va(self, n_class_examples=1900):
        count = 0
        class_item_set_dict = dict()
        item_all = list()

        print('Splitting..')
        while count < n_class_examples * self.config.class_size:
            rand_pick = np.random.randint(len(self.train_data))
            # print(rand_pick)
            label = self.train_data[rand_pick][-1]
            if label in class_item_set_dict:
                item_set = class_item_set_dict[label]
                if len(item_set) < n_class_examples \
                        and rand_pick not in item_set:
                    item_set.add(rand_pick)
                    item_all.append(rand_pick)
                    count += 1
            else:
                class_item_set_dict[label] = set()
                class_item_set_dict[label].add(rand_pick)
                item_all.append(rand_pick)
                count += 1

        train_data2 = list()
        valid_data = list()
        for idx, td in enumerate(self.train_data):
            if idx in item_all:
                valid_data.append(td)
            else:
                train_data2.append(td)

        print(len(train_data2), len(valid_data))
        return train_data2, valid_data
  

In [0]:
args = easydict.EasyDict({
    "train_data_path": 'ag_news_csv/train.csv',
    "test_data_path": 'ag_news_csv/test.csv',
    "pickle_path": 'ag.pkl',
    "seed": 2019,
    "class_size": 4,
    "valid_size_per_class":1000,
    "n_gram": 2,
    "padding":1,
    "max_len":467
})

In [9]:
agdata = load_data(args)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
dictionary size  1439345
Splitting..
116000 4000
train {2: 29000, 3: 29000, 1: 29000, 0: 29000}
test {2: 1900, 3: 1900, 1: 1900, 0: 1900}
real_max_len 465
n_over_max_len 0/116000 (0.0%)


In [10]:
train_data, test_data = agdata.load_csv()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
dictionary size  1439345


In [0]:
train_x = [e[0] for e in train_data]
train_y = [e[2] for e in train_data]
train_x_len = [e[1] for e in train_data]

In [12]:
len(train_x)

120000

In [13]:
max(train_x_len)

465

In [14]:
class_size=4
learning_rate=0.01
batch_size=32
decay_steps=1000
decay_rate=0.9
sequence_length=467
vocab_size = 1439345
embedding_dim = 10
is_training=True
dropout_keep_prob=1
epoch = 5
batch_size = 4096
fastext=FastText(class_size, sequence_length, vocab_size, embedding_dim, learning_rate, decay_steps, epoch)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
fastext.fit(train_x, train_y, batch_size)